<a href="https://colab.research.google.com/github/rdrgzbhnng/marketplace_ai/blob/main/marketplace_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

#Importações e configurações iniciais
import json
import requests
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata

API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=API_KEY)

for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [7]:
# Importar o JSON
json_url = 'https://raw.githubusercontent.com/rdrgzbhnng/marketplace_ai/main/liquidity.json'
response = requests.get(json_url)

if response.status_code == 200:
    # Request successful, proceed to parse JSON data
    json_data = response.json()
else:
    # Handle error (e.g., invalid URL or website down)
    print('Error fetching JSON data:', response.status_code)

# Convertirlo em um DataFrame
liquidity_data = pd.DataFrame(json_data)

# Explorar data
liquidity_data.head()

,ID,From,To,Distance,Time,Date,Type of truck,Weight,Length,Contact
0,1,ES 28001 Madrid,ES 01001 Vitoria-Gasteiz,354km,4h58m,09/05,Box,0.1t,2m,+34 756 56 45
1,2,ES 33001 Oviedo,ES 31001 Pamplona,449km,6h40m,10/05,Box,1.5t,6m,+34 769 59 87
2,3,ES 28001 Madrid,ES 15001 Coruña,592km,8h9m,10/05,Tautliner,24t,13.6m,+34 741 00 23
3,4,ES 28001 Madrid,ES 29330 Almargen,544km,7h42m,10/05,Box,9.5t,6.5m,+34 764 56 88
4,5,ES 44001 Teruel,ES 10001 Cáceres,603km,8h55m,10/05,Tautliner,24t,13.6m,+34 741 82 46


In [28]:
#Embeddando
model = "models/embedding-001"

def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]


#Testando com o primeiro item
first_item = liquidity_data.loc[0]
first_item_id = str(first_item['ID'])
first_item_from = first_item['From']

print(first_item_id)
print(first_item_from)

e = embed_fn(first_item_id, first_item_from)
print(e)

1
ES 28001  Madrid
[0.022305887, 0.010609601, -0.051458623, -0.03026616, 0.087519385, 0.026116319, 0.03694913, -0.037222933, 0.003971513, 0.03386912, -0.02561234, 0.010551982, -0.011823516, 0.03887936, 0.005736713, -0.024599148, 0.019908153, 0.0091823395, -0.017034166, -0.0064524743, 0.029314218, 0.02244109, -0.027147936, 0.0029926235, 0.02718877, -0.012786398, 0.007984082, -0.07358755, -0.03018324, 0.018240152, -0.043467812, 0.023574939, -0.06348584, 0.023574071, 0.006766668, -0.050151818, -0.016142664, 0.024793386, -0.016300255, 0.0135603, 0.02197673, -0.035957694, -0.045727897, -0.007540503, 0.0008003747, 0.013590606, -0.013962878, 0.02374125, 0.017182302, -0.04929718, 0.009318605, -0.012913975, 0.052505136, -0.019939125, -0.0004886851, -0.04085999, 0.043579027, 0.012933173, -0.031007, -0.008924027, 0.0061039045, 0.020696828, 0.033868745, 0.024832683, -0.02422016, -0.07315646, -0.01939641, 0.0044280314, 0.07248691, 0.01787935, -0.01904549, -0.029273858, 0.0385211, -0.007427224, -0.0

In [29]:
#Embeddando geral

liquidity_data["Embeddings"] = liquidity_data.apply(lambda row: embed_fn(str(row["ID"]), row["From"]), axis=1)

#Explorar data
liquidity_data.head()

,ID,From,To,Distance,Time,Date,Type of truck,Weight,Length,Contact,Embeddings
0,1,ES 28001 Madrid,ES 01001 Vitoria-Gasteiz,354km,4h58m,09/05,Box,0.1t,2m,+34 756 56 45,"[0.022305887, 0.010609601, -0.051458623, -0.03..."
1,2,ES 33001 Oviedo,ES 31001 Pamplona,449km,6h40m,10/05,Box,1.5t,6m,+34 769 59 87,"[0.02252799, -0.004548584, -0.05393195, -0.031..."
2,3,ES 28001 Madrid,ES 15001 Coruña,592km,8h9m,10/05,Tautliner,24t,13.6m,+34 741 00 23,"[0.037443545, 0.011773962, -0.048665654, -0.02..."
3,4,ES 28001 Madrid,ES 29330 Almargen,544km,7h42m,10/05,Box,9.5t,6.5m,+34 764 56 88,"[0.045032736, -0.00010115032, -0.05640607, -0...."
4,5,ES 44001 Teruel,ES 10001 Cáceres,603km,8h55m,10/05,Tautliner,24t,13.6m,+34 741 82 46,"[0.030796206, -0.0010795738, -0.053776063, -0...."


In [36]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(liquidity_data["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return liquidity_data.iloc[indice]["ID"]

In [49]:
consulta = "Buscar uma carga desde (from) ES 28001 Madrid"

offer_id = gerar_e_buscar_consulta(consulta, liquidity_data, model)
print(f"Para esta direção de origem, tem um cargamento com o offer_id = {offer_id}")

Para esta direção de origem, tem um cargamento com o offer_id = 29
